# Introduction

In this notebook, we would like to implement the paper [CHRISP](https://link.springer.com/article/10.1007/s10462-020-09833-6). CHRISP is a method that tries to explain the random forest classifier decision making process. In other words, it generates data-driven rules based on what the random forest classifier makes the decisions. So, for instance, suppose we have a data that has only one feature $X \in R$, and two classes $\{0,1\}$. Also suppose that observations with potitive values of X have label $1$, and the ones with negative values of X have label $0$. If we use RandomForest to model this data, a simple decision that can be made by the random forest is that whether X is positive or not. Therefore, in this very simple case, the rule is: 

$ (x>0) \longrightarrow label=1$

# import libraries

In [120]:
import numpy as np
import pandas as pd

from sklearn import datasets 
from sklearn.ensemble import RandomForestClassifier

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

# Implement AdjacentSpaces
**See Algorithm 1 on page 18**

In [3]:
def _get_adjacent_identifier(subspace_range, space_range):
    """
    This function find adjacent spaces of a subspace_range enclosed by space_range
    
    Parameters
    ----------
    
    subspace_range : numpy.ndarray
        has shape (p, 2), where p is the number of dimensions. The first column show lower bounds and the second
        column shows upper bounds.
    
    space_range : numpy.ndarray
        has shape (p, 2), where p is the number of dimensions. The first column show lower bounds and the second
        column shows upper bounds. The space defined by subspace_range is a subset of this space.
    
    Returns
    ----------
        adj_identifier_below : numpy.ndarray
            has shape (p, 2), where the i-th row is the identifer of its corresponding adjacent space, below the subspace 
            
            
        adj_identifier_above: numpy.ndarray
            has shape (p, 2), where the i-th row is the identifer of its corresponding adjacent space, above the subspace
            
    
    NOTE
    ---------
    `adj_identifier` is basically the boundary of one dimensions and the boundaries of other dimensions should be selected
    from subspace range. So, the i-th row of adj-identifier gives boundary of adj space in the i-th dimension, and the 
    boundaries in other dimension for that adj space are the same as what provided in the subsapce range.
    
    
    """
    if subspace_range.shape != space_range.shape:
        raise ValueError("The two inputs must have the same shape.")
        
    if (np.any(subspace_range < space_range[:,0]) 
        or 
        np.any(subspace_range > space_range[:,1])
    ):
        raise ValueError("subspace_range is not fully enclosed by space_range")
        
    
    adj_identifier_below = np.c_[space_range[:,0], subspace_range[:,0]]
    adj_identifier_above = np.c_[subspace_range[:,1], space_range[:,1]]
    
    return adj_identifier_below, adj_identifier_above

In [4]:
space_range = np.array([[-10.0, 10.0],[-5.0, 5.0]], dtype=np.float64)
subspace_range = np.array([[-2.0, 2.0],[-4.0, 4.0]], dtype=np.float64)

adj_identifier_below, adj_identifier_above = _get_adjacent_identifier(subspace_range, space_range)

In [5]:
adj_identifier_below

array([[-10.,  -2.],
       [ -5.,  -4.]])

let us consider the first row of `adj_identifier_below`, i.e., `[-10.,  -2.]`. This is the boundary of the first dimension and the boundary of the other dimension should be obtained from the `subspac_range`. Therefore, the boundary on the second dimension is `subspace_range[1]`, which is `[-4.0, 4.0]`.

Note that the boundary of subspace in the first dimension is `[-2.0, 2.]`. The boundary of the aforementioned adjacent space in the first dimension is `[-10, -2]`. In other words, it is on the left of (below) `[-2.0, 2.]`. We can find the other adjacent spaces similarly.

In [6]:
adj_identifier_above

array([[ 2., 10.],
       [ 4.,  5.]])

# RandomForest Path

The idea is to extract paths in RandomForest, and rank them.

### Random Forest Path Extracting

In [82]:
def get_decision_paths(rf, x, y_threshold=0.5):
    """
    Given the random forest object of sklearn, `rf`, return the decision path of trees that 
    predict the majority vote for each observation
    
    Parameters
    ----------
    rf : object
        A fitted, sklearn-based random-forest classifier
    
    x : a 1D array representing an instance. If 2D, then the first axis must be 1.
    
    y_threshold : the theshold that together with the predicted probability of class of x determines the label.
    
    Returns
    -------
    decision_paths : A list of tuple (j, v, indicator), j is the index of feature, v is the split value, which
    splits the space about that feature and indicator is a binary value where 1 means the j-th feature is lower than
    v.
    """
    x = np.atleast_2d(x.astype(np.float32))
    y_pred = rf.predict_proba(x)
    y_proba_of_x = y_pred[0]  # because x is single observation, y_pred will have one value only
    y = y_proba_of_x[1] # probability of belonging to class 1
    if y < y_threshold:
        y = 0
    else:
        y = 1
    
    decision_paths = []
    for est_idx, est in enumerate(rf.estimators_):
        transactions = []
        y_pred_tree = est.predict(x)
        y_pred_tree = y_pred_tree[0] 
        if y_pred_tree == y:
            DTree_ = est.tree_
            features_id = DTree_.feature
            thresholds_val = DTree_.threshold

            node_indicator = DTree_.decision_path(x)
            leaf_id = DTree_.apply(x)


            sample_id = 0
            node_index = node_indicator.indices[
                node_indicator.indptr[sample_id] : node_indicator.indptr[sample_id + 1]
            ]

            for node_id in node_index:
                # continue to the next node if it is a leaf node
                if leaf_id[sample_id] == node_id:
                    continue

                feature_idx = feature=features_id[node_id]
                threshold = thresholds_val[node_id]
                # check if value of the split feature for sample 0 is below threshold
                if x[sample_id, feature_idx] < threshold:
                    is_below_threshold = 1 # "<"
                else:
                    is_below_threshold = 0 # ">="

                transactions.append((feature, threshold, is_below_threshold))
                
                
            decision_paths.append(transactions)
                
    return decision_paths

In [131]:
iris_data = datasets.load_iris()
X = iris_data['data'].astype(np.float32)
y = iris_data['target']

#to drop class with label y=2
mask = y == 2
X = X[~mask].astype(np.float32)
y = y[~mask]

seed = 0 # for reproducibity
rf = RandomForestClassifier(n_estimators=500, random_state=seed).fit(X, y)

sample_id = 0
x = X[sample_id]
decision_paths = get_decision_path(rf, x)
print('decision_paths: ')
decision_paths

decision_paths: 


[[(3, 0.800000011920929, 1)],
 [(3, 0.800000011920929, 1)],
 [(0, 5.450000047683716, 1), (3, 0.800000011920929, 1)],
 [(1, 3.149999976158142, 0), (0, 5.75, 1)],
 [(3, 0.7000000029802322, 1)],
 [(3, 0.800000011920929, 1)],
 [(3, 0.7000000029802322, 1)],
 [(2, 2.5, 1)],
 [(3, 0.800000011920929, 1)],
 [(2, 2.599999964237213, 1)],
 [(3, 0.800000011920929, 1)],
 [(2, 2.449999988079071, 1)],
 [(3, 0.800000011920929, 1)],
 [(1, 2.950000047683716, 0), (2, 2.9999999403953552, 1)],
 [(2, 2.350000023841858, 1)],
 [(0, 5.450000047683716, 1), (2, 2.449999988079071, 1)],
 [(3, 0.800000011920929, 1)],
 [(0, 5.450000047683716, 1), (3, 0.800000011920929, 1)],
 [(1, 3.049999952316284, 0), (0, 5.8500001430511475, 1)],
 [(2, 2.350000023841858, 1)],
 [(2, 2.350000023841858, 1)],
 [(2, 2.599999964237213, 1)],
 [(2, 2.599999964237213, 1)],
 [(3, 0.800000011920929, 1)],
 [(3, 0.75, 1)],
 [(2, 2.449999988079071, 1)],
 [(3, 0.800000011920929, 1)],
 [(2, 2.449999988079071, 1)],
 [(2, 2.449999988079071, 1)],
 [(2

In [132]:
def get_decision_paths_relax_mapper(decision_paths):
    """
    decision_paths : a list of tuples returned by get_decision_paths()
    output: a dictionary that gives the relaxed threshold for decision (feature_id, indcitor), with n_bin = 1
    """
    lst = []
    for item in decision_paths:
        for triple in item: # triple: tuple with three elements
            lst.append((triple[0], triple[2]))
    
    triples_dict = {item:[] for item in set(lst)}
    for item in decision_paths:
        for triple in item:
            triples_dict[(triple[0], triple[2])].append(triple[1])
    
    for key in triples_dict:
        triples_dict[key] = np.median(triples_dict[key])
        
    return triples_dict


def make_decision_paths_relax(decision_paths):
    triples_relaxer =  get_decision_paths_relax_mapper(decision_paths)
        
    lst = []
    for item in decision_paths:
        tmp = []
        for triple in item:
            tmp.append(
                (
                    triple[0], 
                    triples_relaxer[(triple[0], triple[2])],
                    triple[2]
                )
            )
        
        lst.append(tmp)
    
    return lst

In [133]:
get_decision_paths_relax_mapper(decision_paths)

{(3, 1): 0.800000011920929,
 (1, 0): 3.049999952316284,
 (2, 1): 2.449999988079071,
 (0, 1): 5.450000047683716}

In [134]:
decision_paths_relaxed = make_decision_paths_relax(decision_paths)
decision_paths_relaxed

[[(3, 0.800000011920929, 1)],
 [(3, 0.800000011920929, 1)],
 [(0, 5.450000047683716, 1), (3, 0.800000011920929, 1)],
 [(1, 3.049999952316284, 0), (0, 5.450000047683716, 1)],
 [(3, 0.800000011920929, 1)],
 [(3, 0.800000011920929, 1)],
 [(3, 0.800000011920929, 1)],
 [(2, 2.449999988079071, 1)],
 [(3, 0.800000011920929, 1)],
 [(2, 2.449999988079071, 1)],
 [(3, 0.800000011920929, 1)],
 [(2, 2.449999988079071, 1)],
 [(3, 0.800000011920929, 1)],
 [(1, 3.049999952316284, 0), (2, 2.449999988079071, 1)],
 [(2, 2.449999988079071, 1)],
 [(0, 5.450000047683716, 1), (2, 2.449999988079071, 1)],
 [(3, 0.800000011920929, 1)],
 [(0, 5.450000047683716, 1), (3, 0.800000011920929, 1)],
 [(1, 3.049999952316284, 0), (0, 5.450000047683716, 1)],
 [(2, 2.449999988079071, 1)],
 [(2, 2.449999988079071, 1)],
 [(2, 2.449999988079071, 1)],
 [(2, 2.449999988079071, 1)],
 [(3, 0.800000011920929, 1)],
 [(3, 0.800000011920929, 1)],
 [(2, 2.449999988079071, 1)],
 [(3, 0.800000011920929, 1)],
 [(2, 2.449999988079071, 1)]

In [136]:
transactions = decision_paths_relaxed

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

df.shape

(500, 4)

In [137]:
min_support = 0.25
freq_patterns = apriori(df, min_support=min_support, use_colnames=True)
freq_patterns

,support,itemsets
0,0.462,"((2, 2.449999988079071, 1))"
1,0.506,"((3, 0.800000011920929, 1))"
